<a href="https://colab.research.google.com/github/raultyv/Tareas/blob/main/IAfund_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import math

# --- Datos del Problema ---
CAPACIDAD_CAMION = 700
PESOS_CONTENEDORES = {
    'C1': 100, 'C2': 55, 'C3': 50, 'C4': 112, 'C5': 70,
    'C6': 80, 'C7': 60, 'C8': 118, 'C9': 110, 'C10': 55
}
N_CONTENEDORES = len(PESOS_CONTENEDORES)
NOMBRES_CONTENEDORES = list(PESOS_CONTENEDORES.keys())
VALORES_CONTENEDORES = list(PESOS_CONTENEDORES.values()) # En este problema, valor = peso

# --- Funciones Auxiliares ---

def calcular_fitness(solucion):
    """
    Calcula el peso total de la carga para una solución dada.
    Si excede la capacidad, retorna una penalización negativa.
    """
    peso_total = 0
    for i in range(N_CONTENEDORES):
        if solucion[i] == 1:
            peso_total += VALORES_CONTENEDORES[i] # O PESOS_CONTENEDORES[NOMBRES_CONTENEDORES[i]]

    if peso_total > CAPACIDAD_CAMION:
        return -1  # Penalización: Solución inválida
    return peso_total

def generar_solucion_aleatoria():
    """Genera una solución binaria aleatoria y la ajusta para que sea válida."""
    solucion = [random.randint(0, 1) for _ in range(N_CONTENEDORES)]
    # Ajustar para que sea válida si se excede la capacidad
    while calcular_fitness(solucion) == -1:
        idx_a_quitar = random.choice([i for i, val in enumerate(solucion) if val == 1])
        solucion[idx_a_quitar] = 0
    return solucion

def generar_solucion_greedy():
    """
    Genera una solución inicial usando la heurística greedy (por peso descendente).
    """
    items_ordenados = sorted(
        [(VALORES_CONTENEDORES[i], i) for i in range(N_CONTENEDORES)],
        key=lambda x: x[0],
        reverse=True
    )
    solucion = [0] * N_CONTENEDORES
    peso_actual = 0

    for peso_item, idx_item in items_ordenados:
        if peso_actual + peso_item <= CAPACIDAD_CAMION:
            solucion[idx_item] = 1
            peso_actual += peso_item
    return solucion

def obtener_vecino(solucion_actual):
    """
    Genera una solución vecina aplicando una operación de 'flip' o 'swap'.
    Prioriza 'flip' para añadir un ítem si hay espacio, o quitar uno si hay sobrecarga.
    Si no, hace un 'swap'.
    """
    vecino = list(solucion_actual) # Copia la solución actual
    num_intentos = 0
    max_intentos = 100 # Para evitar bucles infinitos en casos extremos

    while num_intentos < max_intentos:
        # Intento de operación 'flip' (añadir/quitar un solo ítem)
        idx_cambio = random.randint(0, N_CONTENEDORES - 1)
        temp_vecino = list(vecino)

        if temp_vecino[idx_cambio] == 0: # Intenta añadir un ítem
            temp_vecino[idx_cambio] = 1
            if calcular_fitness(temp_vecino) != -1: # Si es válido
                return temp_vecino
        else: # Intenta quitar un ítem
            temp_vecino[idx_cambio] = 0
            # Quitar siempre es válido, pero queremos mejorar o al menos mantener viabilidad
            return temp_vecino

        num_intentos += 1

    # Si después de muchos intentos de 'flip' no se encuentra un vecino válido
    # intentamos un 'swap' si es posible, o simplemente un 'flip' para quitar
    # Si la solución actual es viable, nos enfocamos en no generar una inviable con el swap

    # Intento de operación 'swap' (intercambiar un ítem dentro por uno fuera)
    indices_dentro = [i for i, val in enumerate(solucion_actual) if val == 1]
    indices_fuera = [i for i, val in enumerate(solucion_actual) if val == 0]

    if indices_dentro and indices_fuera:
        idx_quitar = random.choice(indices_dentro)
        idx_anadir = random.choice(indices_fuera)

        vecino_swap = list(solucion_actual)
        vecino_swap[idx_quitar] = 0
        vecino_swap[idx_anadir] = 1

        if calcular_fitness(vecino_swap) != -1:
            return vecino_swap

    # Si todo falla, simplemente devuelve un flip que quita un item para asegurar validez
    if any(solucion_actual): # Si hay algun item para quitar
        idx_quitar = random.choice([i for i, val in enumerate(solucion_actual) if val == 1])
        solucion_final_intento = list(solucion_actual)
        solucion_final_intento[idx_quitar] = 0
        return solucion_final_intento

    # Si la solución está vacía o no se puede generar un vecino válido después de muchos intentos
    # (caso muy raro para este problema), devolver la misma solución
    return solucion_actual


# --- Algoritmo Hill Climbing ---
def hill_climbing():
    print("\n--- Ejecutando Hill Climbing ---")
    start_time = time.time()

    current_solution = generar_solucion_greedy() # Usar greedy como inicio
    # current_solution = generar_solucion_aleatoria() # O iniciar aleatoriamente
    current_fitness = calcular_fitness(current_solution)

    print(f"Inicio - Solución: {[NOMBRES_CONTENEDORES[i] for i, x in enumerate(current_solution) if x == 1]} | Peso: {current_fitness}")

    while True:
        best_neighbor = None
        best_neighbor_fitness = -1

        # Generar múltiples vecinos y elegir el mejor (Estrategia 'Steepest Ascent')
        # Para problemas pequeños, podemos revisar todos los vecinos de un tipo
        # Para problemas grandes, se generaría un subconjunto aleatorio de vecinos

        found_better_neighbor = False

        # Estrategia: Probar 'flips' para mejorar
        for i in range(N_CONTENEDORES):
            temp_neighbor = list(current_solution)
            if temp_neighbor[i] == 0: # Intentar añadir un ítem
                temp_neighbor[i] = 1
                fitness = calcular_fitness(temp_neighbor)
                if fitness > current_fitness:
                    best_neighbor = temp_neighbor
                    best_neighbor_fitness = fitness
                    found_better_neighbor = True
                    #break # Si solo queremos el primer mejor vecino ('First-choice HC')

        # Si ya encontramos una mejora con un flip, podemos detenernos aquí para 'First-choice'
        # o seguir para 'Steepest Ascent' (que es lo que se hace al no poner 'break')

        # Estrategia: Probar 'swaps' si no se encuentra una mejora con 'flips' o para complementar
        if not found_better_neighbor:
            indices_dentro = [i for i, val in enumerate(current_solution) if val == 1]
            indices_fuera = [i for i, val in enumerate(current_solution) if val == 0]

            for i_dentro in indices_dentro:
                for i_fuera in indices_fuera:
                    temp_neighbor = list(current_solution)
                    temp_neighbor[i_dentro] = 0
                    temp_neighbor[i_fuera] = 1
                    fitness = calcular_fitness(temp_neighbor)
                    if fitness > current_fitness:
                        if fitness > best_neighbor_fitness: # Si es mejor que cualquier otro vecino encontrado
                            best_neighbor = temp_neighbor
                            best_neighbor_fitness = fitness
                            found_better_neighbor = True
                            #break # Si solo queremos el primer mejor vecino

        if found_better_neighbor and best_neighbor_fitness > current_fitness:
            current_solution = best_neighbor
            current_fitness = best_neighbor_fitness
            #print(f"Mejora - Solución: {[NOMBRES_CONTENEDORES[i] for i, x in enumerate(current_solution) if x == 1]} | Peso: {current_fitness}")
        else:
            break # No se encontraron mejores vecinos, estamos en un óptimo local

    end_time = time.time()
    execution_time = (end_time - start_time) * 1000 # En milisegundos

    print(f"\nResultado Hill Climbing:")
    print(f"Solución final: {[NOMBRES_CONTENEDORES[i] for i, x in enumerate(current_solution) if x == 1]}")
    print(f"Peso total: {current_fitness} toneladas")
    print(f"Tiempo de ejecución: {execution_time:.4f} ms")
    return current_solution, current_fitness

# --- Algoritmo Simulated Annealing ---
def simulated_annealing(temp_inicial=1000, tasa_enfriamiento=0.99, iteraciones_por_temp=100):
    print("\n--- Ejecutando Simulated Annealing ---")
    start_time = time.time()

    current_solution = generar_solucion_greedy() # Usar greedy como inicio
    # current_solution = generar_solucion_aleatoria() # O iniciar aleatoriamente
    current_fitness = calcular_fitness(current_solution)

    best_solution = list(current_solution)
    best_fitness = current_fitness

    temp = temp_inicial

    # print(f"Inicio - Solución: {[NOMBRES_CONTENEDORES[i] for i, x in enumerate(current_solution) if x == 1]} | Peso: {current_fitness}")

    while temp > 0.1: # La temperatura baja hasta un umbral
        for _ in range(iteraciones_por_temp):
            neighbor_solution = obtener_vecino(current_solution)
            neighbor_fitness = calcular_fitness(neighbor_solution)

            # Asegurarse de que el vecino sea viable (o aplicar penalización)
            if neighbor_fitness == -1:
                continue # Saltar si el vecino es inválido

            delta_e = neighbor_fitness - current_fitness

            if delta_e > 0: # La nueva solución es mejor
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness
                if current_fitness > best_fitness:
                    best_fitness = current_fitness
                    best_solution = list(current_solution)
            else: # La nueva solución es peor o igual
                acceptance_probability = math.exp(delta_e / temp)
                if random.random() < acceptance_probability:
                    current_solution = neighbor_solution
                    current_fitness = neighbor_fitness

        temp *= tasa_enfriamiento # Enfriamiento de la temperatura

    end_time = time.time()
    execution_time = (end_time - start_time) * 1000 # En milisegundos

    print(f"\nResultado Simulated Annealing:")
    print(f"Solución final: {[NOMBRES_CONTENEDORES[i] for i, x in enumerate(best_solution) if x == 1]}")
    print(f"Peso total: {best_fitness} toneladas")
    print(f"Tiempo de ejecución: {execution_time:.4f} ms")
    return best_solution, best_fitness

# --- Ejecución ---
if __name__ == "__main__":
    random.seed(42) # Para reproducibilidad

    # Ejecutar Hill Climbing
    hc_solution, hc_fitness = hill_climbing()

    # Ejecutar Simulated Annealing
    sa_solution, sa_fitness = simulated_annealing()

    print("\n--- Comparación ---")
    print(f"Hill Climbing: {hc_fitness} toneladas")
    print(f"Simulated Annealing: {sa_fitness} toneladas")

    # Calculamos la solución óptima real para comparar
    # Para 10 ítems, la programación dinámica es rápida
    # Esta es solo para verificar, no es parte de la implementación de HC o SA
    dp_table = [[0 for _ in range(CAPACIDAD_CAMION + 1)] for _ in range(N_CONTENEDORES + 1)]

    for i in range(1, N_CONTENEDORES + 1):
        for w in range(1, CAPACIDAD_CAMION + 1):
            peso_actual = VALORES_CONTENEDORES[i-1]
            if peso_actual <= w:
                dp_table[i][w] = max(peso_actual + dp_table[i-1][w - peso_actual], dp_table[i-1][w])
            else:
                dp_table[i][w] = dp_table[i-1][w]

    optimal_fitness = dp_table[N_CONTENEDORES][CAPACIDAD_CAMION]
    print(f"Solución Óptima (Programación Dinámica): {optimal_fitness} toneladas")


--- Ejecutando Hill Climbing ---
Inicio - Solución: ['C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'] | Peso: 700

Resultado Hill Climbing:
Solución final: ['C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
Peso total: 700 toneladas
Tiempo de ejecución: 0.1047 ms

--- Ejecutando Simulated Annealing ---

Resultado Simulated Annealing:
Solución final: ['C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
Peso total: 700 toneladas
Tiempo de ejecución: 475.2297 ms

--- Comparación ---
Hill Climbing: 700 toneladas
Simulated Annealing: 700 toneladas
Solución Óptima (Programación Dinámica): 700 toneladas
